In [54]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [63]:
data = pd.read_excel("dataset_1.xls")

In [69]:
# Remove whitespace from column names
data.columns = list(map(lambda x: x.strip(), data.columns))

# Find values of '  .'
for col in data.columns:
    try:
        data[col][data[col] == '  .'] = np.nan
    except:
        continue

In [70]:
data

,State,Herd,# PTAs,PTAM,PTAF,PTAP,RHA # cows,% W PTAs,RHAM,RHAF,...,Fat10,Pro10,Tdate11,Milk11,Fat11,Pro11,Tdate12,Milk12,Fat12,Pro12
0,11,1,581,272,17,11,590,98,29810,1093,...,3.8,3.1,2017-10-19 00:00:00,80,4,3.1,2017-09-19 00:00:00,78,3.8,3.1
1,11,2,319,309,23,12,331,97,30020,1134,...,.,.,.,NaN,.,.,.,NaN,.,.
2,11,3,10,-448,-18,-19,132,8,22439,892,...,4.2,3.1,2017-08-31 00:00:00,65,3.7,3.1,2017-07-31 00:00:00,64,3.8,3
3,11,4,54,388,16,11,60,90,23466,821,...,3.4,2.9,.,NaN,.,.,.,NaN,.,.
4,11,5,76,658,21,19,80,95,27769,1118,...,4.1,3,2017-09-14 00:00:00,74,4.1,3.1,2017-08-16 00:00:00,72,4,3
5,11,6,38,20,-4,1,75,51,16129,589,...,4.1,3,2017-10-10 00:00:00,43,3.8,3,2017-09-12 00:00:00,41,4,2.9
6,11,7,111,-15,0,-1,176,63,22194,805,...,3.3,3.1,.,NaN,.,.,.,NaN,.,.
7,11,8,152,362,19,15,210,72,28424,1044,...,3.9,3.2,2017-10-06 00:00:00,76,3.8,3,2017-09-06 00:00:00,75,3.8,3
8,11,9,41,311,2,8,48,86,21075,763,...,3.3,3.2,2017-10-12 00:00:00,54,4,3,2017-09-12 00:00:00,64,3.3,3
9,11,10,37,-21,7,-1,51,72,16093,591,...,4.2,3.3,2017-10-10 00:00:00,43,3.8,3.1,2017-09-08 00:00:00,46,3.5,3.1


In [43]:
data["State"][data["State"] == "  ."] = np.nan

/home/brian/.local/lib/python3.6/site-packages/pandas/core/ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


TypeError: invalid type comparison

In [58]:
data["Fat10"][data["Fat10"] == "  ."] = np.nan

In [66]:
data.Fat10[1]

' . '